In [1]:
import mimicfouretl.bigquery_utils as bq

### Example Usage with BigQuery Connection
Requires users to get access to Physionet data 

In [2]:
bq.set_credentials_file('../bq_credentials/client_secret.json')
# bq.set_project_id('mimic-iv-418015')
bq.set_project_id('micro-vine-412020')

In [3]:
client = bq.get_client(use_service_account_auth=True)
tables = bq.list_tables('mimiciv_icu', client)
# tables = bq.list_tables(client, 'physionet-data.mimiciv_hosp')
tables

['mimiciv_icu.caregiver',
 'mimiciv_icu.chartevents',
 'mimiciv_icu.d_items',
 'mimiciv_icu.datetimeevents',
 'mimiciv_icu.icustays',
 'mimiciv_icu.ingredientevents',
 'mimiciv_icu.inputevents',
 'mimiciv_icu.outputevents',
 'mimiciv_icu.procedureevents']

In [4]:
spark = bq.get_spark_session(use_service_account_auth=True)

24/03/28 21:20:14 WARN Utils: Your hostname, Dunyas-Laptop.local resolves to a loopback address: 127.0.0.1; using 10.0.0.117 instead (on interface en0)
24/03/28 21:20:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/dunyaoguz/.ivy2/cache
The jars for the packages stored in: /Users/dunyaoguz/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-82f2b250-1fda-44eb-bf38-8843a850d08b;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/dunyaoguz/anaconda3/envs/mimic-iv-etl/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.37.0 in central
	[0.37.0] com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;latest.version
:: resolution report :: resolve 1143ms :: artifacts dl 3ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.37.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   1   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-82f2b250-1fda-44eb-bf38-8843a850d08b
	confs: [default]
	0 artifacts copied, 1 already retrieved (0kB/6ms)
24/03/28 21:20:15 WARN NativeCodeLoader: Unable to load native-hadoo

In [6]:
results = bq.run_query(spark, "SELECT * FROM `mimiciv_hosp.admissions` LIMIT 10")

### Example Usage with Local Connection
Use a local copy of a sample of physionet data

In [3]:
spark = bq.get_spark_session(use_local_data=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/28 21:23:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
tables = bq.list_tables('mimiciv_icu', spark, use_local_data=True)
tables

['mimiciv_icu.d_items',
 'mimiciv_icu.inputevents',
 'mimiciv_icu.chartevents',
 'mimiciv_icu.icustays',
 'mimiciv_icu.datetimeevents',
 'mimiciv_icu.outputevents',
 'mimiciv_icu.ingredientevents',
 'mimiciv_icu.procedureevents']

In [5]:
results = bq.run_query(spark, "SELECT * FROM `admissions` LIMIT 10", use_local_data=True)

hcpcsevents
d_items
patients
emar
d_hcpcs
pharmacy
emar_detail


24/03/28 21:23:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


inputevents
d_labitems
chartevents
icustays
prescriptions
diagnoses_icd
transfers
d_icd_diagnoses
datetimeevents
outputevents
poe
ingredientevents
omr
procedures_icd
microbiologyevents
poe_detail
procedureevents
labevents
services
d_icd_procedures
admissions


In [6]:
results.toPandas()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,12039279,22872837,2188-04-05 14:36:00,2188-04-07 12:24:00,NaT,URGENT,P138WD,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,MARRIED,OTHER,NaT,NaT,0
1,11119137,23598731,2110-12-06 09:15:00,2110-12-10 17:30:00,NaT,SURGICAL SAME DAY ADMISSION,P147HV,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,SINGLE,WHITE,NaT,NaT,0
2,12568379,29461608,2200-12-21 15:20:00,2200-12-22 15:59:00,NaT,EW EMER.,P367GD,EMERGENCY ROOM,HOME,Medicare,ENGLISH,SINGLE,WHITE,2200-12-21 12:53:00,2200-12-21 16:52:00,0
3,12568379,22444607,2203-05-01 17:02:00,2203-05-02 11:40:00,NaT,AMBULATORY OBSERVATION,P25DU0,PACU,None,Medicare,ENGLISH,SINGLE,WHITE,NaT,NaT,0
4,12270967,23436743,2130-02-24 21:14:00,2130-02-28 15:50:00,NaT,EW EMER.,P93E1Z,PACU,HOME,Other,ENGLISH,MARRIED,WHITE,NaT,NaT,0
5,10222281,25236487,2117-08-19 03:18:00,2117-08-25 15:27:00,NaT,OBSERVATION ADMIT,P68URZ,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2117-08-18 17:36:00,2117-08-19 04:17:00,0
6,10222281,22582267,2116-05-06 02:33:00,2116-05-12 15:16:00,NaT,SURGICAL SAME DAY ADMISSION,P345XU,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,NaT,NaT,0
7,11574058,22636689,2150-10-04 04:56:00,2150-10-04 10:56:00,NaT,EW EMER.,P85DBF,EMERGENCY ROOM,AGAINST ADVICE,Other,ENGLISH,MARRIED,WHITE,2150-10-04 00:51:00,2150-10-04 06:02:00,0
8,16589694,26285610,2152-12-04 16:04:00,2152-12-07 12:48:00,NaT,OBSERVATION ADMIT,P75S70,WALK-IN/SELF REFERRAL,REHAB,Medicare,ENGLISH,WIDOWED,WHITE,2152-12-04 08:59:00,2152-12-04 17:23:00,0
9,14718466,27697843,2163-11-20 21:46:00,2163-11-28 14:20:00,NaT,EW EMER.,P100U2,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,ENGLISH,WIDOWED,WHITE,2163-11-20 17:18:00,2163-11-20 23:00:00,0


24/03/28 23:52:56 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 950010 ms exceeds timeout 120000 ms
24/03/28 23:52:56 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/29 00:08:01 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$